<a href="https://colab.research.google.com/github/AgnusSerena/MentalHealthChatbot/blob/main/Mental_Health_Chatbot_GEN_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install langchain_groq langchain_core langchain_community

In [11]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
result = llm.invoke("Who is lord Ram?")
print(result.content)

Lord Rama is a major deity in Hinduism and is considered one of the most important figures in Hindu mythology. He is the seventh avatar (incarnation) of the god Vishnu and is revered as a symbol of righteousness, duty, and virtue.

According to the Hindu epic, the Ramayana, Lord Rama was a king of Ayodhya, a city in ancient India. He was born to King Dasharatha and Queen Kausalya, and was the eldest of four brothers, including Lakshmana, Bharata, and Shatrughna.

The story of Lord Rama is as follows:

* Lord Rama was exiled to the forest for 14 years by his stepmother, Queen Kaikeyi, who wanted her own son, Bharata, to become the king.
* During his exile, Lord Rama's wife, Sita, was abducted by the demon king Ravana, who took her to his kingdom in Lanka (modern-day Sri Lanka).
* Lord Rama, along with his brother Lakshmana and the monkey god Hanuman, embarked on a journey to rescue Sita and defeat Ravana.
* After a long and arduous battle, Lord Rama defeated Ravana and rescued Sita, ret

In [12]:
!pip install pypdf

In [13]:

!pip install chromadb

In [14]:

!pip install sentence_transformers

In [15]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
def initialize_llm():
  llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
  return llm
def create_vector_db():
  loader = DirectoryLoader("/content/data/", glob = '*.pdf', loader_cls = PyPDFLoader)
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma.from_documents(texts, embeddings, persist_directory = './chroma_db')
  vector_db.persist()

  print("ChromaDB created and data saved")

  return vector_db

def setup_qa_chain(vector_db, llm):
  retriever = vector_db.as_retriever()
  prompt_templates = """ You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
    {context}
    User: {question}
    Chatbot: """
  PROMPT = PromptTemplate(template = prompt_templates, input_variables = ['context', 'question'])

  qa_chain = RetrievalQA.from_chain_type(
      llm = llm,
      chain_type = "stuff",
      retriever = retriever,
      chain_type_kwargs = {"prompt": PROMPT}
  )
  return qa_chain
def main():
  print("Intializing Chatbot.........")
  llm = initialize_llm()

  db_path = "/content/chroma_db"

  if not os.path.exists(db_path):
    vector_db  = create_vector_db()
  else:
    embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
  qa_chain = setup_qa_chain(vector_db, llm)

  while True:
    query = input("\nHuman: ")
    if query.lower()  == "exit":
      print("Chatbot: Take Care of yourself, Goodbye!")
      break
    response = qa_chain.run(query)
    print(f"Chatbot: {response}")

if __name__ == "__main__":
  main()


Intializing Chatbot.........


<ipython-input-15-3642f106f843>:53: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)



Human: Hii
Chatbot: Hello! It seems like you've shared a passage about mental health, its definition, and its importance, especially in adolescents. The text highlights the complexity of mental health, emphasizing that it's not just about individual well-being but also about the social and physical environments that influence our lives.

It's interesting to note that the definition of mental health has evolved over time, and different organizations, like the World Health Organization (WHO), have contributed to our understanding of the concept. The passage also mentions the importance of considering the capacity of individuals and groups to interact with each other and their environment in ways that promote subjective well-being.

I'm here to listen and support you. What are your thoughts on this passage? Is there something specific that resonated with you, or would you like to discuss mental health in general? I'm all ears!

Human: I am very depressed
Chatbot: I'm so sorry to hear tha

In [16]:
create_vector_db()

ChromaDB created and data saved


In [17]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 61.7 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [18]:

from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import gradio as gr
def initialize_llm():
  llm = ChatGroq(
  temperature = 0,
  groq_api_key = "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",
  model_name = "llama-3.3-70b-versatile"
)
  return llm

def create_vector_db():
  loader = DirectoryLoader("/content/data/", glob = '*.pdf', loader_cls = PyPDFLoader)
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma.from_documents(texts, embeddings, persist_directory = './chroma_db')
  vector_db.persist()

  print("ChromaDB created and data saved")

  return vector_db

def setup_qa_chain(vector_db, llm):
  retriever = vector_db.as_retriever()
  prompt_templates = """ You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
    {context}
    User: {question}
    Chatbot: """
  PROMPT = PromptTemplate(template = prompt_templates, input_variables = ['context', 'question'])

  qa_chain = RetrievalQA.from_chain_type(
      llm = llm,
      chain_type = "stuff",
      retriever = retriever,
      chain_type_kwargs = {"prompt": PROMPT}
  )
  return qa_chain

llm = initialize_llm()

db_path = "/content/chroma_db"

if os.path.exists(db_path):
    print("✅ Loading existing ChromaDB...")
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory='./chroma_db', embedding_function=embeddings)  # ✅ Load existing DB
else:
    print("🚀 Creating a new ChromaDB...")
    vector_db = create_vector_db()  # ✅ Create new DB

qa_chain = setup_qa_chain(vector_db, llm)

def chatbot_response(user_input, history = []):
  print("Chat bot running...")
  if not user_input.strip():
    return "Please provide a valid input", history
  response = qa_chain.run(user_input)
  print(f"Chatbot: {response}")
  history.append((user_input, response))
  return "", history

with gr.Blocks(theme = 'gr.themes.Soft()') as app:
  gr.Markdown("# 🧠 Mental Health Chatbot 🤖")
  gr.Markdown("A compassionate chatbot designed to assist with mental well-being. Please note: For serious concerns, contact a professional.")

  chatbot = gr.Interface(fn=chatbot_response, title="Mental Health Chatbot", inputs = "text", outputs="text")

  gr.Markdown("This chatbot provides general support. For urgent issues, seek help from licensed professionals.")

app.launch()

✅ Loading existing ChromaDB...


/usr/local/lib/python3.11/dist-packages/gradio/blocks.py:1057: UserWarning: Cannot load gr.themes.Soft(). Caught Exception: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: 'gr.themes.Soft()'.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://46c83202f85f36feb0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
